# Problem might lie in loss! 
# Noooooo! Problem is not the loss! 

In [20]:
import torch
import torch.nn as nn

In [43]:
ls = nn.MSELoss(reduction='none')

In [44]:
mask = torch.tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 0.],
        [1., 0., 0.]])

In [72]:
x = torch.rand(128, 3, 2)
y = torch.rand(128, 3, 2)
mask = torch.ones(128, 3)

In [73]:
loss = ls(x, y)

In [74]:
loss.shape

torch.Size([128, 3, 2])

In [75]:
mask.shape

torch.Size([128, 3])

In [76]:
b, t, f = y.shape

In [77]:
maskex = mask.unsqueeze(-1).expand((b, t, f)).bool()

In [78]:
maskex

tensor([[[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]],

      

In [79]:
loss.masked_select(maskex).mean()

tensor(0.1741)

In [80]:
y_hat_masked = y.masked_select(maskex)
y_masked = x.masked_select(maskex)

In [81]:
loss_fn = nn.MSELoss(reduction='mean')

In [82]:
loss = loss_fn(y_hat_masked, y_masked)

In [83]:
loss

tensor(0.1741)

In [84]:
loss_fn = nn.MSELoss(reduction='none')

In [85]:
def masked_loss(loss_fn, y_hat, y, mask):
    """
    Computes the masked loss given a loss function, predicted values, ground truth values, and a mask tensor.
    """
    b, t, f = y_hat.shape
    mask = mask.unsqueeze(-1).expand((b, t, f)).bool()  # both y_hat and y are of same size
    # Apply mask to the predicted and ground truth values
    y_hat_masked = y_hat.masked_select(mask)
    y_masked = y.masked_select(mask)

    # Calculate the loss using the masked values
    # loss = loss_fn(y_hat_masked, y_masked)
    loss = torch.sum(loss_fn(y_hat_masked, y_masked)) / torch.sum(mask)

    return loss

In [86]:
masked_loss(loss_fn, y, x, mask)

tensor(0.1741)

In [87]:
torch.sum(mask)

tensor(384.)

In [2]:
import librosa
import librosa.display
import numpy as np

In [3]:
y, sr = librosa.load('s2201b_00000935.wav', sr = None)

In [2]:

normalizedy = librosa.util.normalize(y)

stft = librosa.core.stft(normalizedy, n_fft = 256, hop_length=16)
mel = librosa.feature.melspectrogram(S=stft, n_mels=32)
mellog = np.log(mel + 1e-9)
melnormalized = librosa.util.normalize(mellog)